# Dynamics 365 Business Central Trouble Shooting Guide (TSG) - Extensions

This notebook contains Kusto queries that can help getting to the root cause of an issue with extensions for one or more environments. 

NB! Some of the signal used in this notebook is only available in newer versions of Business Central, so check the version of your environment if some sections do not return any data. The signal documentation states in which version a given signal was introduced.

## 1\. Get setup: Load up Python libraries and connect to Application Insights

First you need to set the notebook Kernel to Python3, load the KQLmagic module (did you install it?) and connect to your Application Insights resource (get appid and appkey from the API access page in the Application Insights portal)

In [13]:
# load the KQLmagic module
%reload_ext Kqlmagic

In [14]:
# Connect to the Application Insights API
%kql appinsights://appid='<add app id from the Application Insights portal>';appkey='<add API key from the Application Insights portal>'

## 2\. Define filters

This workbook is designed for troubleshooting extensions. Please provide values for aadTenantId, environmentName, and extensionId (or use a config file).

You can also specify limits to the period of time that the analysis should include.

In [21]:
# Add values for AAD tenant id, environment name, and extension id.
# It is possible to leave one or more values blank (if you want to analyze across all values of the parameter)

# You can either use configuration file (INI file format) or set filters directly. 

# If you specify a config file, then variables set here takes precedence over manually set filter variables
# config file name and directory (full path)
configFile = "c:/tmp/notebook.ini"


# Add AAD tenant id and environment name here
aadTenantId = "MyaaDtenantId"
environmentName = "MyEnvironmentName"
extensionId = "MyExtensionId"

# date filters for the analysis
# use YYYY-MM-DD format for the dates (ISO 8601)
startDate = "2020-11-20"
endDate = "2020-11-24"




# Do not edit this code section
import configparser
config = configparser.ConfigParser()

config.read(configFile)

if bool(config.defaults()):
    if config.has_option('DEFAULT', 'aadTenantId'):
        aadTenantId = config['DEFAULT']['aadTenantId']
    if config.has_option('DEFAULT', 'environmentName'):
        environmentName = config['DEFAULT']['environmentName']
    if config.has_option('DEFAULT', 'extensionId'):
        extensionId = config['DEFAULT']['extensionId']
    if config.has_option('DEFAULT', 'startDate'):    
        startDate = config['DEFAULT']['startDate']
    if config.has_option('DEFAULT', 'endDate'):
        endDate = config['DEFAULT']['endDate']

print("Using these parameters for the analysis:")
print("----------------------------------------")
print("aadTenantId         " + aadTenantId)
print("environmentName     " + environmentName)
print("extensionId         " + extensionId)
print("startDate           " + startDate)
print("endDate             " + endDate)

Using these parameters for the analysis:
----------------------------------------
aadTenantId         0f3ec54f-5f3e-432a-8c80-0eae667293e7
environmentName     
extensionId         MyExtensionId
startDate           2020-11-01
endDate             2020-12-01


# Analyze extension events
Now you can run Kusto queries to look for possible root causes for issues about extensions.

Either click **Run All** above to run all sections, or scroll down to the type of analysis you want to do and manually run queries

## Extension event overview

Event telemetry docs: 
* https://docs.microsoft.com/en-us/dynamics365/business-central/dev-itpro/administration/telemetry-extension-lifecycle-trace
* https://docs.microsoft.com/en-us/dynamics365/business-central/dev-itpro/administration/telemetry-extension-update-trace

KQL samples: https://github.com/microsoft/BCTech/blob/master/samples/AppInsights/KQL/RawData/ExtensionLifecycle.kql

In [16]:
%%kql
//
// extension event types stats 
// 
let _aadTenantId = aadTenantId;
let _environmentName = environmentName;
let _extensionId = extensionId;
let _startDate = startDate;
let _endDate = endDate;
traces
| where 1==1 
    and timestamp >= todatetime(_startDate)
    and timestamp <= todatetime(_endDate) + totimespan(24h) - totimespan(1ms)
    and (_aadTenantId == '' or customDimensions.aadTenantId == _aadTenantId)
    and (_environmentName == '' or customDimensions.environmentName == _environmentName )
    and (_extensionId == '' or customDimensions.extensionId == _extensionId)    
    and customDimensions.eventId in ('RT0010', 'LC0010', 'LC0011', 'LC0012', 'LC0013', 'LC0014', 'LC0015', 'LC0016', 'LC0017', 'LC0018', 'LC0019', 'LC020', 'LC021', 'LC022', 'LC023')    
| extend aadTenantId=tostring( customDimensions.aadTenantId)
       , environmentName=tostring( customDimensions.environmentName )
       , extensionId=tostring( customDimensions.extensionId )
       , eventId=tostring(customDimensions.eventId)
| extend eventMessageShort=case(
              eventId=='RT0010', strcat(eventId, ': Extension Update Failed (exception raised in upgrade code)')
            , eventId=='LC0010', strcat(eventId, ': Extension installed successfully')
            , eventId=='LC0011', strcat(eventId, ': Extension failed to install')
            , eventId=='LC0012', strcat(eventId, ': Extension synchronized successfully')            
            , eventId=='LC0013', strcat(eventId, ': Extension failed to synchronize')            
            , eventId=='LC0014', strcat(eventId, ': Extension published successfully')            
            , eventId=='LC0015', strcat(eventId, ': Extension failed to publish')            
            , eventId=='LC0016', strcat(eventId, ': Extension un-installed successfully')            
            , eventId=='LC0017', strcat(eventId, ': Extension failed to un-install')            
            , eventId=='LC0018', strcat(eventId, ': Extension unpublished successfully')
            , eventId=='LC0019', strcat(eventId, ': Extension failed to un-publish')
            , eventId=='LC0020', strcat(eventId, ': Extension compiled successfully')
            , eventId=='LC0021', strcat(eventId, ': Extension failed to compile')
            , eventId=='LC0022', strcat(eventId, ': Extension updated successfully')
            , eventId=='LC0023', strcat(eventId, ': Extension failed to update')
            , strcat( eventId, ': Unknown message')
         )
| summarize count() by eventMessageShort
| order by eventMessageShort
| render barchart with (title='Extension lifecycle event overview', legend=hidden)

In [17]:
%%kql
//
// top 100 extension events
// 
let _aadTenantId = aadTenantId;
let _environmentName = environmentName;
let _extensionId = extensionId;
let _startDate = startDate;
let _endDate = endDate;
traces
| where 1==1 
    and timestamp >= todatetime(_startDate)
    and timestamp <= todatetime(_endDate) + totimespan(24h) - totimespan(1ms)
    and (_aadTenantId == '' or customDimensions.aadTenantId == _aadTenantId)
    and (_environmentName == '' or customDimensions.environmentName == _environmentName )
    and (_extensionId == '' or customDimensions.extensionId == _extensionId)    
    and customDimensions.eventId in ('RT0010', 'LC0010', 'LC0011', 'LC0012', 'LC0013', 'LC0014', 'LC0015', 'LC0016', 'LC0017', 'LC0018', 'LC0019', 'LC020', 'LC021', 'LC022', 'LC023')    
| extend aadTenantId=tostring( customDimensions.aadTenantId)
       , environmentName=tostring( customDimensions.environmentName )
       , extensionId=tostring( customDimensions.extensionId )
       , extensionName=tostring( customDimensions.extensionName )
| limit 100
| project aadTenantId, environmentName, extensionId, extensionName, timestamp, message
| order by aadTenantId asc, environmentName asc, extensionId asc, timestamp desc

## Extension failures

Event telemetry docs: 
* https://docs.microsoft.com/en-us/dynamics365/business-central/dev-itpro/administration/telemetry-extension-lifecycle-trace
* https://docs.microsoft.com/en-us/dynamics365/business-central/dev-itpro/administration/telemetry-extension-update-trace

In [18]:
%%kql
//
// extension event failure overview
// 
let _aadTenantId = aadTenantId;
let _environmentName = environmentName;
let _extensionId = extensionId;
let _startDate = startDate;
let _endDate = endDate;
traces
| where 1==1 
    and timestamp >= todatetime(_startDate)
    and timestamp <= todatetime(_endDate) + totimespan(24h) - totimespan(1ms)
    and (_aadTenantId == '' or customDimensions.aadTenantId == _aadTenantId)
    and (_environmentName == '' or customDimensions.environmentName == _environmentName )
    and (_extensionId == '' or customDimensions.extensionId == _extensionId)    
    and customDimensions.eventId in ('RT0010', 'LC0011', 'LC0013', 'LC0015', 'LC0017', 'LC0019', 'LC021', 'LC023')    
| extend aadTenantId=tostring( customDimensions.aadTenantId)
       , environmentName=tostring( customDimensions.environmentName )
       , extensionId=tostring( customDimensions.extensionId )
       , eventId=tostring(customDimensions.eventId)
| extend eventMessageShort=case(
              eventId=='RT0010', strcat(eventId, ': Extension Update Failed (exception raised in upgrade code)')
            , eventId=='LC0011', strcat(eventId, ': Extension failed to install')
            , eventId=='LC0013', strcat(eventId, ': Extension failed to synchronize')            
            , eventId=='LC0015', strcat(eventId, ': Extension failed to publish')            
            , eventId=='LC0017', strcat(eventId, ': Extension failed to un-install')            
            , eventId=='LC0019', strcat(eventId, ': Extension failed to un-publish')
            , eventId=='LC0021', strcat(eventId, ': Extension failed to compile')
            , eventId=='LC0023', strcat(eventId, ': Extension failed to update')
            , strcat( eventId, ': Unknown message')
         )
| summarize count() by eventMessageShort
| order by eventMessageShort
| render columnchart with (title='Failure type overview', xtitle="", legend=hidden)

In [19]:
%%kql
//
// top 100 extension event failure details
// 
let _aadTenantId = aadTenantId;
let _environmentName = environmentName;
let _extensionId = extensionId;
let _startDate = startDate;
let _endDate = endDate;
traces
| where 1==1 
    and timestamp >= todatetime(_startDate)
    and timestamp <= todatetime(_endDate) + totimespan(24h) - totimespan(1ms)
    and (_aadTenantId == '' or customDimensions.aadTenantId == _aadTenantId)
    and (_environmentName == '' or customDimensions.environmentName == _environmentName )
    and (_extensionId == '' or customDimensions.extensionId == _extensionId)
    and customDimensions.eventId in ('RT0010', 'LC0011', 'LC0013', 'LC0015', 'LC0017', 'LC0019', 'LC021', 'LC023')    
| extend aadTenantId=tostring( customDimensions.aadTenantId)
       , environmentName=tostring( customDimensions.environmentName )
       , extensionId=tostring( customDimensions.extensionId )
       , eventId=tostring(customDimensions.eventId)
       , extensionName=tostring(customDimensions.extensionName)
| extend eventMessageShort=case(
              eventId=='RT0010', strcat(eventId, ': Extension Update Failed (exception raised in upgrade code)')
            , eventId=='LC0011', strcat(eventId, ': Extension failed to install')
            , eventId=='LC0013', strcat(eventId, ': Extension failed to synchronize')            
            , eventId=='LC0015', strcat(eventId, ': Extension failed to publish')            
            , eventId=='LC0017', strcat(eventId, ': Extension failed to un-install')            
            , eventId=='LC0019', strcat(eventId, ': Extension failed to un-publish')
            , eventId=='LC0021', strcat(eventId, ': Extension failed to compile')
            , eventId=='LC0023', strcat(eventId, ': Extension failed to update')
            , strcat( eventId, ': Unknown message')
         )
| project aadTenantId, environmentName, extensionId, extensionName, eventMessageShort, failureReason=customDimensions.failureReason, alStackTrace=customDimensions.alStackTrace
| limit 100